# Fine Tunning Mistral-7B + LoRA

Instalo librerias necesarias

In [2]:
pip install torch transformers datasets peft bitsandbytes accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Importo todas las librerias necesarias

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import torch

Configuro el modelo y tokenizadr. Indico tambien el quantificador 4bit para ahorrar memoria.

In [3]:
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

#LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


Cargo el dataset del notebook del repo. Lo convierto en formato hf

In [4]:
def download_and_standardize(dataset_name, rename_columns=None):
    print(f"Downloading {dataset_name}...")
    dataset = load_dataset(dataset_name)

    if "train" in dataset:
        data = dataset["train"]
    else:
        data = dataset[list(dataset.keys())[0]]

    data_list = [dict(row) for row in data]

    if rename_columns:
        for row in data_list:
            for old_key, new_key in rename_columns.items():
                if old_key in row:
                    row[new_key] = row.pop(old_key)

    for row in data_list:
        row.setdefault("context", "")
        row.setdefault("question", "")
        row.setdefault("answer", "")

    return data_list

datasets_info = [
    {"name": "ahmed000000000/cybersec", "rename_columns": {"INSTRUCTION": "question", "RESPONSE": "answer"}},
    {"name": "dzakwan/cybersec", "rename_columns": {"instruction": "context", "input": "question", "output": "answer"}},
    {"name": "asimsultan/cyber2k", "rename_columns": None},
    {"name": "Vanessasml/cybersecurity_32k_instruction_input_output", "rename_columns": {"instruction": "context", "input": "question", "output": "answer"}}
]

final_dataset = []
for dataset in datasets_info:
    data_list = download_and_standardize(dataset["name"], dataset["rename_columns"])
    final_dataset.extend(data_list)

train_phi.jsonl:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12408 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

seed_generation-14k.json:   0%|          | 0.00/10.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13982 [00:00<?, ? examples/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65232 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/3.00k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.47M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32569 [00:00<?, ? examples/s]

In [5]:
from datasets import Dataset
hf_dataset = Dataset.from_list(final_dataset)

In [20]:
hf_dataset.shape

(124191, 3)

In [21]:
hf_dataset

Dataset({
    features: ['question', 'answer', 'context'],
    num_rows: 124191
})

In [6]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

hf_dataset_train_test = hf_dataset.train_test_split(test_size=0.2, seed=420, shuffle = True)

hf_dataset_val_test = hf_dataset_train_test["test"].train_test_split(test_size=0.5, seed=420, shuffle = True)

split_hf_dataset = DatasetDict({
    "train" : hf_dataset_train_test["train"],
    "val" : hf_dataset_val_test["train"],
    "test" : hf_dataset_val_test["test"]
})

print(split_hf_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 99352
    })
    val: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 12419
    })
    test: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 12420
    })
})


Para comprobar que todo funciona reduzco con esta funcion propia el tamaño del dataset

In [11]:
def sample_dataset_with_proportions(dataset_dict, total_sample_size):
    total_examples = sum(len(dataset_dict[split]) for split in dataset_dict.keys())

    sampled_dataset = DatasetDict()

    for split in dataset_dict.keys():
        split_proportion = len(dataset_dict[split]) / total_examples

        split_sample_size = min(int(total_sample_size * split_proportion), len(dataset_dict[split]))

        sampled_dataset[split] = dataset_dict[split].shuffle(seed=420).select(range(split_sample_size))

    return sampled_dataset

total_sample_size = 1000
reduced_dataset = sample_dataset_with_proportions(split_hf_dataset, total_sample_size)

for split, data in reduced_dataset.items():
    print(f"{split}: {len(data)} examples")


train: 799 examples
val: 99 examples
test: 100 examples


Defino el prompt de entrenamiento

In [12]:
def format_prompt(example):
    return f"""Eres un experto en ciberseguridad para usuarios no técnicos. Responde de forma clara y sencilla.

### Instrucción:
{example['question']}

### Contexto:
{example['context']}

### Respuesta:
{example['answer']}"""


Configuro los prametros de entrenamiento

In [15]:
training_args = TrainingArguments(
    output_dir="./mistral-7b-ciberseguridad",
    num_train_epochs=3, #3
    per_device_train_batch_size=6, #6
    gradient_accumulation_steps=3,  #3
    gradient_checkpointing=True,
    learning_rate=0.5,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    bf16=True,
    logging_steps=20,
    save_strategy="steps",
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=reduced_dataset["train"],
    eval_dataset=reduced_dataset["val"],
    #max_seq_length=2048,
    formatting_func=format_prompt,
    args=training_args
)

<ipython-input-15-a7ca63d0f3f1>:19: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying formatting function to train dataset:   0%|          | 0/799 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/799 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/799 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/799 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Entreno y guardo el modelo

In [16]:
trainer.train()
trainer.model.save_pretrained("./mistral-7b-ciberseguridad-lora")
tokenizer.save_pretrained("./mistral-7b-ciberseguridad-lora")

Step,Training Loss
20,9.286500
40,7.627700
60,7.891800
80,7.452000
100,8.152700
120,7.441800


('./mistral-7b-ciberseguridad-lora/tokenizer_config.json',
 './mistral-7b-ciberseguridad-lora/special_tokens_map.json',
 './mistral-7b-ciberseguridad-lora/tokenizer.model',
 './mistral-7b-ciberseguridad-lora/added_tokens.json',
 './mistral-7b-ciberseguridad-lora/tokenizer.json')

Ejemplo de uso del modelo

In [18]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, "./mistral-7b-ciberseguridad-lora")
model = model.merge_and_unload()

prompt = """Eres un asistente de ciberseguridad. Explica de forma simple:
¿Qué es un ataque de phishing y cómo puedo evitarlo?"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    inputs=inputs.input_ids,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Eres un asistente de ciberseguridad. Explica de forma simple:
¿Qué es un ataque de phishing y cómo puedo evitarlo?

### What is a phishing attack?

Phishing attacks are a type of cybercrime that involves the use of fraudulent emails or websites to trick people into providing sensitive information, such as passwords, bank account details, or personal information. The goal of a phishing attack is to gain access to this information and use it to commit fraud or steal money.

### How can I protect myself from phishing attacks?

Here are a few tips to help you protect yourself from phishing attacks:

1. Be wary of emails or websites that ask for your personal information, especially if they come from unknown sources.
2. Never click on links or download attachments in emails unless you are certain of their authenticity.
3. Use strong, unique passwords for all of your accounts and change them regularly.
4. Keep your antivirus and security software up to date.
5. Be suspicious of emails or web